# Lecture 1: Rcpp

In [1]:
library(Rcpp)

## 1

In [2]:
cppFunction('
int one() {
    return 1;
}
')

x <- one()

In [3]:
cppFunction('
double sumC(NumericVector x) {
    int n = x.size();
    double total = 0;
    int i;
    for (i = 0; i < n; i++) {
        total = total + x[i];
    }
    return total;
}')

test = 1:100
sumC(test)

[1] 5050

In [5]:
cppFunction('
double my_mean(NumericVector x) {
    int n = x.size();
    double mean = 0;
    int i;
    for (i = 0; i < n; i++) {
        mean = mean + x[i];
    }
    mean = mean / n;
    return mean;
}')

my_mean(c(1,2,3))

[1] 2

In [6]:
cppFunction('
NumericVector my_check_C(NumericVector x, double c) {
    int n = x.size();


    // this works the same way as python
    // clone must be used for this otherwise
    // the variable points to the same element in memory
    NumericVector out = clone(x);
    int i;

    for (i = 0; i < n; i++) {
        if (x[i] <= c) {
            out[i] = 0;
        }
    }
    return out;
}')

my_check_C(c(1,3,5,5,3,2,2,4,3,4,7,6,4,3,5,5), 3)

[1] 0 0 5 5 0 0 0 4 0 4 7 6 4 0 5 5

In [7]:
cppFunction('
NumericVector my_cumsum(NumericVector x) {
    int n = x.size();

    NumericVector out = clone(x);
    double total = 0;
    int i;

    for (i = 0; i < n; i++) {
        total = total + x[i];
        out[i] = total;
    }
    return out;
}')

my_cumsum(1:10)


[1]  1  3  6 10 15 21 28 36 45 55

In [8]:
cppFunction('
NumericVector my_range(NumericVector x) {    
    NumericVector out(2);
    out[1] = max(x);
    out[0] = min(x);
    return out;
}')

my_range(c(3,1,5,2))

[1] 1 5

In [10]:
cppFunction('
NumericVector my_range_manual(NumericVector x) {    
    NumericVector out(2);
    NumericVector minmax(2);
    int n = x.size();
    int i;

    double maximum = 0;
    double minimum = 0;

    for (i = 0; i < n; i++) {
        minmax[0] = x[i];
        minmax[1] = x[i];
        if (minmax[0] > x[i]) {
            out[0] = x[i];
        }
        if (minmax[1] < x[i]) {
            out[1] = x[i];
        }
    }
    return out;
}')

my_range(c(3,1,5,2,2,3,4,55,6,2,4,3,2,24,1,343,5,4,23,12,32,3,23,23,332,3,2,3,2,-1))

[1]  -1 343

In [11]:
cppFunction('
NumericVector rowSd_C(NumericMatrix mt) {
    int nrow = mt.nrow();
    int ncol = mt.ncol();

    NumericVector means(nrow);
    NumericVector out(nrow);
    double total;
    int i,j;

    //loop for rows
    for (i = 0; i < nrow; i++) {
        total = 0;
        //Sugar for computing the row means
        means[i] = mean(mt(i,_));
        //loop for columns
        for (j = 0; j < ncol; j++) {
            // pow is for x^y
            total = total + pow(mt(i,j) - means[i], 2.0);
        }
        out[i] = sqrt(total / (ncol-1));
    }
    return out;
}')

In [12]:
cppFunction('
NumericVector rowSd_C(NumericMatrix mt) {
    int nrow = mt.nrow();
    int ncol = mt.ncol();

    NumericVector out(nrow);
    double total;
    int i,j;

    //loop for rows
    for (i = 0; i < nrow; i++) {
        out[i] = sd(mt(i,_));
    }
    return out;
}')